In [ ]:
import json
from tensorflow import keras

eval_data_url = "https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json"
eval_path = keras.utils.get_file("eval.json", eval_data_url)


with open(eval_path) as f:
    raw_eval_data = json.load(f)

4857856/4854279 [==============================] - 0s 0us/step


In [ ]:
corpus = []

for i in range( len(raw_eval_data["data"]) ):
    for j in range( len(raw_eval_data["data"][i]["paragraphs"])):

        corpus.append(raw_eval_data["data"][i]["paragraphs"][j]["context"])
        

In [ ]:
import spacy
spcy = spacy.load('en_core_web_sm')

In [ ]:
import re

def preprocess(string):

    sentence = string.lower() 

    sentence = re.sub('[^a-z]',' ',sentence )
    sentence = re.sub(' +',' ',sentence)

    sentence = spcy(sentence)

    token_list = []

    for token in sentence:
        token_list.append(token)

    sentence = ([token.lemma_.lower() for token in token_list])

    return sentence

In [ ]:
processed = []

for i in corpus:
    processed.append(preprocess(i))

In [ ]:
!pip install rank_bm25

from rank_bm25 import BM25Okapi

bm25 = BM25Okapi(processed)

In [ ]:
query = "Who correct most previous misunderstandings of motion and force?"
query = preprocess(query)
bm25.get_top_n(query, corpus, n=1)

['Philosophers in antiquity used the concept of force in the study of stationary and moving objects and simple machines, but thinkers such as Aristotle and Archimedes retained fundamental errors in understanding force. In part this was due to an incomplete understanding of the sometimes non-obvious force of friction, and a consequently inadequate view of the nature of natural motion. A fundamental error was the belief that a force is required to maintain motion, even at a constant velocity. Most of the previous misunderstandings about motion and force were eventually corrected by Galileo Galilei and Sir Isaac Newton. With his mathematical insight, Sir Isaac Newton formulated laws of motion that were not improved-on for nearly three hundred years. By the early 20th century, Einstein developed a theory of relativity that correctly predicted the action of forces on objects with increasing momenta near the speed of light, and also provided insight into the forces produced by gravitation an

In [ ]:
!pip install tokenizers
!pip install transformers

     |████████████████████████████████| 3.3MB 3.5MB/s 
     |████████████████████████████████| 2.1MB 3.6MB/s 
     |████████████████████████████████| 901kB 15.4MB/s 


In [ ]:
import os
import re
import json
import string
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer, TFBertModel, BertConfig

max_len = 384
configuration = BertConfig()

In [ ]:
slow_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
save_path = "bert_base_uncased/"
if not os.path.exists(save_path):
    os.makedirs(save_path)
slow_tokenizer.save_pretrained(save_path)

# Load the fast tokenizer from saved file
tokenizer = BertWordPieceTokenizer("bert_base_uncased/vocab.txt", lowercase=True)

In [ ]:
def create_model():

    encoder = TFBertModel.from_pretrained("bert-base-uncased")

    input_ids = layers.Input(shape=(max_len,), dtype=tf.int32)
    token_type_ids = layers.Input(shape=(max_len,), dtype=tf.int32)
    attention_mask = layers.Input(shape=(max_len,), dtype=tf.int32)
    embedding = encoder(
        input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask
    )[0]

    start_logits = layers.Dense(1, name="start_logit", use_bias=False)(embedding)
    start_logits = layers.Flatten()(start_logits)

    end_logits = layers.Dense(1, name="end_logit", use_bias=False)(embedding)
    end_logits = layers.Flatten()(end_logits)

    start_probs = layers.Activation(keras.activations.softmax)(start_logits)
    end_probs = layers.Activation(keras.activations.softmax)(end_logits)

    model = keras.Model(
        inputs=[input_ids, token_type_ids, attention_mask],
        outputs=[start_probs, end_probs],
    )
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    optimizer = keras.optimizers.Adam(lr=5e-5)
    model.compile(optimizer=optimizer, loss=[loss, loss])
    return model

In [ ]:
model = create_model()

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model.load_weights("/content/drive/MyDrive/NLP Project/single_span/singlespan.h5")

In [ ]:
def create_input(question, context):

    context = " ".join(str(context).split())
    question = " ".join(str(question).split())

    tokenized_context = tokenizer.encode(context)
    tokenized_question = tokenizer.encode(question)


    input_ids = tokenized_context.ids + tokenized_question.ids[1:]
    token_type_ids = [0] * len(tokenized_context.ids) + [1] * len(tokenized_question.ids[1:])

    attention_mask = [1] * len(input_ids)

    padding_length = max_len - len(input_ids)

    if padding_length > 0:  
        input_ids = input_ids + ([0] * padding_length)
        attention_mask = attention_mask + ([0] * padding_length)
        token_type_ids = token_type_ids + ([0] * padding_length)

    return [np.array([input_ids]), np.array([token_type_ids]), np.array([attention_mask])] , tokenized_context.offsets    

In [ ]:
a, offsets = create_input("who is fast", "jim is not fast. john doe is fast")

In [ ]:
model_output = model.predict(a)

In [ ]:
def get_answer(model_output, offsets, context):

    start_pred, end_pred = model_output

    start = np.argmax(start_pred)
    end = np.argmax(end_pred)
    

    if(start > end):
        end = np.argmax(end_pred[:,start:]) + start

    pred_char_start = offsets[start][0]
    pred_char_end = offsets[end][1]
    pred_ans = context[pred_char_start:pred_char_end]

    return pred_ans



In [ ]:
def ask_question(question):

    query = preprocess(question)
    context = bm25.get_top_n(query, corpus, n=1)[0]

    input, offsets = create_input(question, context)
    output = model.predict(input)

    answer = get_answer(output, offsets, context)

    print("answer:",answer)
    print("context from corpus:", context)

In [ ]:
ask_question("how many natural reserves are there in warsaw")

answer: 13
context from corpus: There are 13 natural reserves in Warsaw – among others, Bielany Forest, Kabaty Woods, Czerniaków Lake. About 15 kilometres (9 miles) from Warsaw, the Vistula river's environment changes strikingly and features a perfectly preserved ecosystem, with a habitat of animals that includes the otter, beaver and hundreds of bird species. There are also several lakes in Warsaw – mainly the oxbow lakes, like Czerniaków Lake, the lakes in the Łazienki or Wilanów Parks, Kamionek Lake. There are lot of small lakes in the parks, but only a few are permanent – the majority are emptied before winter to clean them of plants and sediments.


In [ ]:
ask_question("who developed the theory of relativity")

answer: Einstein
context from corpus: Tesla was generally antagonistic towards theories about the conversion of matter into energy.:247 He was also critical of Einstein's theory of relativity, saying:


In [ ]:
ask_question("which is the largest florida city")

answer: Jacksonville
context from corpus: Jacksonville is the largest city by population in the U.S. state of Florida, and the largest city by area in the contiguous United States. It is the county seat of Duval County, with which the city government consolidated in 1968. Consolidation gave Jacksonville its great size and placed most of its metropolitan population within the city limits; with an estimated population of 853,382 in 2014, it is the most populous city proper in Florida and the Southeast, and the 12th most populous in the United States. Jacksonville is the principal city in the Jacksonville metropolitan area, with a population of 1,345,596 in 2010.


In [ ]:
ask_question("when was warsaw occupied by germany")

answer: 4 August 1915 until November 1918
context from corpus: Warsaw was occupied by Germany from 4 August 1915 until November 1918. The Allied Armistice terms required in Article 12 that Germany withdraw from areas controlled by Russia in 1914, which included Warsaw. Germany did so, and underground leader Piłsudski returned to Warsaw on 11 November and set up what became the Second Polish Republic, with Warsaw the capital. In the course of the Polish-Bolshevik War of 1920, the huge Battle of Warsaw was fought on the eastern outskirts of the city in which the capital was successfully defended and the Red Army defeated. Poland stopped by itself the full brunt of the Red Army and defeated an idea of the "export of the revolution".


In [ ]:
ask_question("how many general principles are declared by the social charter")

answer: 30
context from corpus: The Social Chapter is a chapter of the 1997 Treaty of Amsterdam covering social policy issues in European Union law. The basis for the Social Chapter was developed in 1989 by the "social partners" representatives, namely UNICE, the employers' confederation, the European Trade Union Confederation (ETUC) and CEEP, the European Centre of Public Enterprises. A toned down version was adopted as the Social Charter at the 1989 Strasbourg European Council. The Social Charter declares 30 general principles, including on fair remuneration of employment, health and safety at work, rights of disabled and elderly, the rights of workers, on vocational training and improvements of living conditions. The Social Charter became the basis for European Community legislation on these issues in 40 pieces of legislation.


In [ ]:
ask_question("which market replaced the butcher market")


answer: Grainger Market
context from corpus: The Grainger Market replaced an earlier market originally built in 1808 called the Butcher Market. The Grainger Market itself, was opened in 1835 and was Newcastle's first indoor market. At the time of its opening in 1835 it was said to be one of the largest and most beautiful markets in Europe. The opening was celebrated with a grand dinner attended by 2000 guests, and the Laing Art Gallery has a painting of this event. With the exception of the timber roof which was destroyed by a fire in 1901 and replaced by latticed-steel arches the Market is largely in its original condition. The Grainger Market architecture, like most in Grainger Town, which are either grade I or II listed, was listed grade I in 1954 by English Heritage.


In [ ]:
ask_question("which church condemns capital punishment")

answer: The United Methodist Church
context from corpus: The United Methodist Church, along with other Methodist churches, condemns capital punishment, saying that it cannot accept retribution or social vengeance as a reason for taking human life. The Church also holds that the death penalty falls unfairly and unequally upon marginalized persons including the poor, the uneducated, ethnic and religious minorities, and persons with mental and emotional illnesses. The United Methodist Church also believes that Jesus explicitly repudiated the lex talionis in Matthew 5:38-39 and abolished the death penalty in John 8:7. The General Conference of the United Methodist Church calls for its bishops to uphold opposition to capital punishment and for governments to enact an immediate moratorium on carrying out the death penalty sentence.


In [ ]:
ask_question("what is the yuan dynasty officially known as")

answer: Great Yuan
context from corpus: The Yuan dynasty (Chinese: 元朝; pinyin: Yuán Cháo), officially the Great Yuan (Chinese: 大元; pinyin: Dà Yuán; Mongolian: Yehe Yuan Ulus[a]), was the empire or ruling dynasty of China established by Kublai Khan, leader of the Mongolian Borjigin clan. Although the Mongols had ruled territories including today's North China for decades, it was not until 1271 that Kublai Khan officially proclaimed the dynasty in the traditional Chinese style. His realm was, by this point, isolated from the other khanates and controlled most of present-day China and its surrounding areas, including modern Mongolia and Korea. It was the first foreign dynasty to rule all of China and lasted until 1368, after which its Genghisid rulers returned to their Mongolian homeland and continued to rule the Northern Yuan dynasty. Some of the Mongolian Emperors of the Yuan mastered the Chinese language, while others only used their native language (i.e. Mongolian) and the 'Phags-pa s

In [ ]:
ask_question("who supplied four of eight songs of the First Lutheran hymnal Achtliederbuch")

answer: Luther
context from corpus: Luther's hymns were included in early Lutheran hymnals and spread the ideas of the Reformation. He supplied four of eight songs of the First Lutheran hymnal Achtliederbuch, 18 of 26 songs of the Erfurt Enchiridion, and 24 of the 32 songs in the first choral hymnal with settings by Johann Walter, Eyn geystlich Gesangk Buchleyn, all published in 1524.


In [ ]:
def create_corpus_from_file(filename):

    with open(filename, 'r') as f:

        string = f.read()

    corpus = string.split('//')

    for i in range(len(corpus)):
        corpus[i] = re.sub('\n',' ',corpus[i])
        if(corpus[i][0] == ' '):
            corpus[i] = corpus[i][1:]

    return corpus

In [ ]:
corpus = create_corpus_from_file('corpus.txt')

processed = []

for i in corpus:
    processed.append(preprocess(i))

In [ ]:
bm25 = BM25Okapi(processed)

In [ ]:
ask_question("what does sulphur dioxide cause")

answer: irritation to the eyes
context from corpus: Oxides of Sulphur: Oxides of sulphur are produced when sulphur containing fossil fuel is burnt. The most common species, sulphur dioxide, is a gas that is poisonous to both animals and plants. It has been reported that even a low concentration of sulphur dioxide causes respiratory diseases e.g., asthma, bronchitis, emphysema in human beings. Sulphur dioxide causes irritation to the eyes, resulting in tears and redness. High concentration of SO2 leads to stiffness of flower buds which eventually fall off from plants. Uncatalysed oxidation of sulphur dioxide is slow. However, the presence of particulate matter in polluted air catalyses the oxidation of sulphur dioxide to sulphur trioxide. The reaction can also be promoted by ozone and hydrogen peroxide. 


In [ ]:
ask_question("how does troposphere pollution occur")

answer: due to the presence of undesirable solid or gaseous particles in the air
context from corpus: Tropospheric pollution occurs due to the presence of undesirable solid or gaseous particles in the air. The following are the major gaseous and particulate pollutants present in the troposphere: 1. Gaseous air pollutants: These are oxides of sulphur, nitrogen and carbon, hydrogen sulphide, hydrocarbons, ozone and other oxidants. 2. Particulate pollutants: These are dust, mist, fumes, smoke, smog etc. 


In [ ]:
ask_question("what is the color of haze caused by oxides of nitrogen")

answer: red
context from corpus: The irritant red haze in the traffic and congested places is due to oxides of nitrogen. Higher concentrations of NO2 damage the leaves of plants and retard the rate of photosynthesis. Nitrogen dioxide is a lung irritant that can lead to an acute respiratory disease in children. It is toxic to living tissues also. Nitrogen dioxide is also harmful to various textile fibres and metals. 


In [ ]:
ask_question("what has increased due to  use of chemical fertilizers and the burning of fossil fuels")

answer: Nitrous oxide
context from corpus: Besides carbon dioxide, other greenhouse gases are methane, water vapour, nitrous oxide, CFCs and ozone. Methane is produced naturally when vegetation is burnt, digested or rotted in the absence of oxygen. Large amounts of methane are released in paddy fields, coal mines, from rotting garbage dumps and by fossil fuels. Chlorofluorocarbons (CFCs) are man-made industrial chemicals used in air conditioning etc. CFCs are also damaging the ozone layer (Section 14.2.2). Nitrous oxide occurs naturally in the environment. In recent years, their quantities have increased significantly due to the use of chemical fertilizers and the burning of fossil fuels. If these trends continue, the average global temperature will increase to a level which may lead to melting of polar ice caps and flooding of low lying areas all over the earth. Increase in the global temperature increases the incidence of infectious diseases like dengue, malaria, yellow fever, sleepi

In [ ]:
ask_question("what happens when global temperature increases") #vague answer

answer: Increase in the global temperature increases the incidence of infectious diseases
context from corpus: Besides carbon dioxide, other greenhouse gases are methane, water vapour, nitrous oxide, CFCs and ozone. Methane is produced naturally when vegetation is burnt, digested or rotted in the absence of oxygen. Large amounts of methane are released in paddy fields, coal mines, from rotting garbage dumps and by fossil fuels. Chlorofluorocarbons (CFCs) are man-made industrial chemicals used in air conditioning etc. CFCs are also damaging the ozone layer (Section 14.2.2). Nitrous oxide occurs naturally in the environment. In recent years, their quantities have increased significantly due to the use of chemical fertilizers and the burning of fossil fuels. If these trends continue, the average global temperature will increase to a level which may lead to melting of polar ice caps and flooding of low lying areas all over the earth. Increase in the global temperature increases the inciden

In [ ]:
ask_question("why is carbon monoxide poisonous") #vague answer

answer: It binds to haemoglobin to form carboxyhaemoglobin
context from corpus: Carbon monoxide (CO) is one of the most serious air pollutants. It is a colourless and odourless gas, highly poisonous to living beings because of its ability to block the delivery of oxygen to the organs and tissues. It is produced as a result of incomplete combustion of carbon. Carbon monoxide is mainly released into the air by automobile exhaust. Other sources, which produce CO, involve incomplete combustion of coal, firewood, petrol, etc. The number of vehicles has been increasing over the years all over the world. Many vehicles are poorly maintained and several have inadequate pollution control equipments resulting in the release of greater amount of carbon monoxide and other polluting gases. Do you know why carbon monoxide is poisonous? It binds to haemoglobin to form carboxyhaemoglobin, which is about 300 times more stable than the oxygen-haemoglobin complex. In blood, when the concentration of carbo

In [ ]:
ask_question("what is the pH of rain water")

answer: 5.6
context from corpus: We are aware that normally rain water has a pH of 5.6 due to the presence of H+ ions formed by the reaction of rain water with carbon dioxide present in the atmosphere. When the pH of the rain water drops below 5.6, it is called acid rain. Acid rain refers to the ways in which acid from the atmosphere is deposited on the earth’s surface. Oxides of nitrogen and sulphur which are acidic in nature can be blown by wind along with solid particles in the atmosphere and finally settle down either on the ground as dry deposition or in water, fog and snow as wet deposition. 


In [ ]:
ask_question("why is the ph of rain water 5.6")

answer: H+ ions formed by the reaction of rain water with carbon dioxide present in the atmosphere
context from corpus: We are aware that normally rain water has a pH of 5.6 due to the presence of H+ ions formed by the reaction of rain water with carbon dioxide present in the atmosphere. When the pH of the rain water drops below 5.6, it is called acid rain. Acid rain refers to the ways in which acid from the atmosphere is deposited on the earth’s surface. Oxides of nitrogen and sulphur which are acidic in nature can be blown by wind along with solid particles in the atmosphere and finally settle down either on the ground as dry deposition or in water, fog and snow as wet deposition. 


In [ ]:
ask_question("what is acid rain")

answer: rain water has a pH of 5.6 due to the presence of H+ ions formed by the reaction of rain water with carbon dioxide present in the atmosphere. When the pH of the rain water drops below 5.6, it is called acid rain. Acid rain refers to the ways in which acid from the atmosphere is deposited on the earth’s surface
context from corpus: We are aware that normally rain water has a pH of 5.6 due to the presence of H+ ions formed by the reaction of rain water with carbon dioxide present in the atmosphere. When the pH of the rain water drops below 5.6, it is called acid rain. Acid rain refers to the ways in which acid from the atmosphere is deposited on the earth’s surface. Oxides of nitrogen and sulphur which are acidic in nature can be blown by wind along with solid particles in the atmosphere and finally settle down either on the ground as dry deposition or in water, fog and snow as wet deposition. 


In [ ]:
ask_question("which Indian monument has been affected by acid rain")

answer: Taj Mahal
context from corpus: Acid rain is harmful for agriculture, trees and plants as it dissolves and washes away nutrients needed for their growth. It causes respiratory ailments in human beings and animals. When acid rain falls and flows as ground water to reach rivers, lakes etc. it affects plants and animal life in aquatic ecosystem. It corrodes water pipes resulting in the leaching of heavy metals such as iron, lead and copper into the drinking water. Acid rain damages buildings and other structures made of stone or metal. The Taj Mahal in India has been affected by acid rain. 


In [ ]:
ask_question("what is smog")

answer: It is a mixture of smoke, fog and sulphur dioxide
context from corpus: The word smog is derived from smoke and fog. This is the most common example of air pollution that occurs in many cities throughout the world. There are two types of smog: (a) Classical smog occurs in cool humid climate. It is a mixture of smoke, fog and sulphur dioxide. Chemically it is a reducing mixture and so it is also called as reducing smog. (b) Photochemical smog occurs in warm, dry and sunny climate. The main components of the photochemical smog result from the action of sunlight on unsaturated hydrocarbons and nitrogen oxides produced by automobiles and factories. Photochemical smog has high concentration of oxidising agents and is, therefore, called as oxidising smog. 


In [ ]:
ask_question("what gases cause greenhouse effect") #not complete

answer: Carbon dioxide
context from corpus: We all know that in cold places flowers, vegetables and fruits are grown in glass covered areas called greenhouse. Do you know that we humans also live in a greenhouse? Of course, we are not surrounded by glass but a blanket of air called the atmosphere, which has kept the temperature on earth constant for centuries. But it is now undergoing change, though slowly. Just as the glass in a greenhouse holds the sun’s warmth inside, atmosphere traps the sun’s heat near the earth’s surface and keeps it warm. This is called natural greenhouse effect because it maintains the temperature and makes the earth perfect for life. In a greenhouse, solar radiations pass through the transparent glass and heat up the soil and the plants. The warm soil and plants emit infrared radiations. Since glass is opaque to infrared radiations (thermal region), it partly reflects and partly absorbs these radiations. This mechanism keeps the energy of the sun trapped in th

In [ ]:
ask_question("what is the ozone hole") #wrong answer


answer: ozone hole
context from corpus: In 1980s atmospheric scientists working in Antarctica reported about depletion of ozone layer commonly known as ozone hole over the South Pole. It was found that a unique set of conditions was responsible for the ozone hole. In summer season, nitrogen dioxide and methane react with chlorine monoxide (reaction iv) and chlorine atoms (reaction v) forming chlorine sinks, preventing much ozone depletion, whereas in winter, special type of clouds called polar stratospheric clouds are formed over Antarctica. These polar stratospheric clouds provide surface on which chlorine nitrate formed (reaction iv) gets hydrolysed to form hypochlorous acid (reaction (vi)). It also reacts with hydrogen chloride produced as per reaction (v) to give molecular chlorine. When sunlight returns to the Antarctica in the spring, the sun’s warmth breaks up the clouds and HOCl and Cl2 are photolysed by sunlight, as given in reactions (viii) and (ix). The chlorine radicals thu

In [ ]:
ask_question("what is green chemistry")

answer: a production process that would bring about minimum pollution or deterioration to the environment
context from corpus: The solution of this problem does not lie in stopping the process of development that has been set in; but to discover methods, which would help in the reduction of deterioration of the environment. Green chemistry is a way of thinking and is about utilising the existing knowledge and principles of chemistry and other sciences to reduce the adverse impact on environment. Green chemistry is a production process that would bring about minimum pollution or deterioration to the environment. The byproducts generated during a process, if not used gainfully, add to the environmental pollution. Such processes are not only environmental unfriendly but also cost-ineffective. The waste generation and its disposal both are economically unsound. Utilisation of existing knowledge base for reducing the chemical hazards along with the developmental activities is the foundation

In [ ]:
ask_question("who launched the clean india mission")

answer: the Government of India
context from corpus: You must be aware of the ‘Swachh Bharat Abhiyan’ or ‘Clean India Mission’ launched by the Government of India. 


In [ ]:
ask_question("what is O3 called")

answer: ozone
context from corpus: The upper stratosphere consists of considerable amount of ozone (O3), which protects us from the harmful ultraviolet (UV) radiations (λ 255 nm) coming from the sun. These radiations cause skin cancer (melanoma) in humans. Therefore, it is important to maintain the ozone shield. Ozone in the stratosphere is a product of UV radiations acting on dioxygen (O2) molecules. The UV radiations split apart molecular oxygen into free oxygen (O) atoms. These oxygen atoms combine with the molecular oxygen to form ozone. 


In [ ]:
ask_question("what is the source of harmful ultraviolet radiations")

answer: the sun
context from corpus: The upper stratosphere consists of considerable amount of ozone (O3), which protects us from the harmful ultraviolet (UV) radiations (λ 255 nm) coming from the sun. These radiations cause skin cancer (melanoma) in humans. Therefore, it is important to maintain the ozone shield. Ozone in the stratosphere is a product of UV radiations acting on dioxygen (O2) molecules. The UV radiations split apart molecular oxygen into free oxygen (O) atoms. These oxygen atoms combine with the molecular oxygen to form ozone. 
